In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../input/palmer-archipelago-antarctica-penguin-data/penguins_size.csv")

In [ ]:
df

In [ ]:
df.info()

## EDA

### Missing Data


In [ ]:
df.isna().sum()

In [ ]:
df.describe().T

In [ ]:
sns.kdeplot(df['culmen_length_mm'])

In [ ]:
sns.kdeplot(df['culmen_depth_mm'])

In [ ]:
sns.kdeplot(df['flipper_length_mm'])

In [ ]:
sns.kdeplot(df['body_mass_g'])

In [ ]:
df['body_mass_g'] = df['body_mass_g'].fillna(df['body_mass_g'].median())

In [ ]:
df.isna().sum()

In [ ]:
df[['culmen_length_mm' , 'culmen_depth_mm' , 'flipper_length_mm']] = df[['culmen_length_mm' , 'culmen_depth_mm' , 'flipper_length_mm']].fillna(df[['culmen_length_mm' , 'culmen_depth_mm' , 'flipper_length_mm']].mean())

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
df['sex'].value_counts(dropna=False)

In [ ]:
df = df[~(df['sex'] == '.')]

In [ ]:
df.groupby('sex')['body_mass_g'].min()

In [ ]:
df[(df['body_mass_g'] > 3900) & (df['sex'] == 'FEMALE')]

In [ ]:
df[df['sex'].isna()]

In [ ]:
def determine_sex(row):
    if row['body_mass_g'] > 3900:
        return 'MALE'
    else:
        return 'FEMALE'

In [ ]:
df['sex'] = df.apply(lambda row: determine_sex(
    row) if pd.isna(row['sex']) else row['sex'], axis=1)

In [ ]:
df[(df['body_mass_g'] > 3900) & (df['sex'] == 'FEMALE')]

In [ ]:
df.groupby('sex')['body_mass_g'].mean()

In [ ]:
df.isna().sum()

## Visualization


In [ ]:
df.head()

In [ ]:
sp = df['species'].value_counts()

In [ ]:
sns.barplot(x=sp.index, y=sp.values)

In [ ]:
sns.scatterplot(x='culmen_length_mm', y='culmen_depth_mm',
                data=df, hue='species', palette='Dark2')

In [ ]:
sns.pairplot(df, hue='species', palette='Dark2')

In [ ]:
sns.catplot(x='species', y='culmen_length_mm', data=df,
            kind='box', col='sex', palette='Dark2')

In [ ]:
sns.catplot(x='species', y='body_mass_g', data=df,
            kind='box', col='sex', palette='Dark2')

## Feature Engineering


In [ ]:
df.head()

In [ ]:
X = pd.get_dummies(df.drop('species', axis=1), drop_first=True)

In [ ]:
X

In [ ]:
y = df['species']

## Train | Test Split


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101, stratify=y)

In [ ]:
y.value_counts(normalize=True)

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts(normalize=True)

# Decision Tree Classifier

## Default Hyperparameters


In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
base_pred = model.predict(X_test)

In [ ]:
base_pred

## Evaluation


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(y_test, base_pred)

In [ ]:
con = ConfusionMatrixDisplay(cm, display_labels=model.classes_)
con.plot()

In [ ]:
print(classification_report(y_test, base_pred))

In [ ]:
X.head()

In [ ]:
model.feature_importances_

In [ ]:
pd.DataFrame(index=X.columns, data=model.feature_importances_,
             columns=['Feature Importance'])

## Visualize the Tree


In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)
plot_tree(model, feature_names=X.columns, filled=True)

## Reporting Model Results

To begin experimenting with hyperparameters, let's create a function that reports back classification results and plots out the tree.


In [ ]:
def report_model(model):
    model_preds = model.predict(X_test)
    print(classification_report(y_test, model_preds))
    print('\n')
    plt.figure(figsize=(12, 8), dpi=150)
    plot_tree(model, filled=True, feature_names=X.columns)

In [ ]:
report_model(model)

In [ ]:
pruned_tree = DecisionTreeClassifier(max_depth=2)
pruned_tree.fit(X_train, y_train)

In [ ]:
report_model(pruned_tree)

In [ ]:
pruned_tree = DecisionTreeClassifier(max_leaf_nodes=3)
pruned_tree.fit(X_train, y_train)

In [ ]:
report_model(pruned_tree)

In [ ]:
entropy_tree = DecisionTreeClassifier(criterion='entropy')
entropy_tree.fit(X_train, y_train)

In [ ]:
report_model(entropy_tree)